<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [2]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname2.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

4554752/4553829 [==============================] - 0s 0us/step


In [3]:
vocab = sorted(set(corpus))
print(len(vocab), "chars")

48 chars


In [4]:
print(vocab)

['\t', '\n', ' ', '(', ')', '«', '»', '،', '؟', 'ء', 'آ', 'أ', 'ؤ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'ٔ', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '\u200c']


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [0]:
seq_length = 1000
examples_per_epoch = len(corpus)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>

# Model (GRU)

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
def create_model_gru(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE):
  model = keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
  model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [14]:
gru_model = create_model_gru()
gru_model.summary()
gru_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_5 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense_2 (Dense)              (64, None, 48)            49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = gru_model.fit(dataset, epochs=50)

Epoch 1/50
42/42 [==============================] - 110s 3s/step - loss: 11.2932 - accuracy: 0.2322
Epoch 2/50
42/42 [==============================] - 110s 3s/step - loss: 2.4952 - accuracy: 0.3244
Epoch 3/50
42/42 [==============================] - 110s 3s/step - loss: 2.2354 - accuracy: 0.3522
Epoch 4/50
42/42 [==============================] - 111s 3s/step - loss: 2.1602 - accuracy: 0.3667
Epoch 5/50
42/42 [==============================] - 111s 3s/step - loss: 2.1044 - accuracy: 0.3886
Epoch 6/50
42/42 [==============================] - 111s 3s/step - loss: 2.0445 - accuracy: 0.4075
Epoch 7/50
42/42 [==============================] - 111s 3s/step - loss: 1.9892 - accuracy: 0.4274
Epoch 8/50
42/42 [==============================] - 111s 3s/step - loss: 1.9345 - accuracy: 0.4437
Epoch 9/50
42/42 [==============================] - 111s 3s/step - loss: 1.8794 - accuracy: 0.4595
Epoch 10/50
42/42 [==============================] - 111s 3s/step - loss: 1.8411 - accuracy: 0.4716
Epoch 11

In [16]:
main_gru_model = create_model_gru(batch_size=1)
main_gru_model.set_weights(gru_model.get_weights())
main_gru_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_3 (Dense)              (1, None, 48)             49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [0]:
# main_gru_model = tf.keras.models.load_model('char_gru.h5')
# main_gru_model.summary()

In [18]:
print(generate_text(main_gru_model, start_string=u"که ایران چوباغیست خرم بهار", temperature=1))

که ایران چوباغیست خرم بهار 
 به من بد تو را چه یک بی‌گناه 	 که بسیار هدیه و فره آمی‌وفا 
 کزین چاره جای او شد چیست 	 از آتش بدو گفت پرنازدید 
 بمانش همه غام را گفت ناکه منزو 	 ورا در جنگ گشت زرد و مرد دلیر 
 شترویش با ماه انشین را به ز چین 	 کسی را که دانار یل مرگ خشتش افروز 
 به شمشیر برخاست با خان به کشته دست 	 دل من بپیچد یکسر مه دید 
 تو پی تو بگذشا هزاران بخورد 	 زمین و کوس و به زهر و بلند 
 بزن گونه شادی باش و کنند 	 برو تا تخمهٔ ماه هور مهم 
 نبینی که آفرین گرد کس 	 بچنگاه بازی دل از دارد ز طوس 
 که رستم گشت ختر بسیچید باد 	 بجستند با او کجا سرو بر من ز گرگ 
 بپوشید بشد از دود بر چشم 	 ز هر کس به دیرایگی من مکن 
 هرآنکس که مغز وز هنگ شد 	 سخنهای بزرگی همی ز آژگاه 
 برفتند سینده روم باشی همی 	 همی کرد جهان را نشانم شیر 
 چنین گفت کز میغ اسفندیار 	 و گر نیز صد از رود و هر جوی 
 بیامد ز خشم سوار گذشت 	 که چند ز فران و هوشید تباه پیل 
 تبه خوانیه فریدون شبست 	 ده سرش اسب جنگ گفتار ماند 
 گساوان زرین رزمگاه آم دان 	 به توم آن سو رودگان رزمند 
 شوم بنموند بر پیلتن 	 که آن رامش اندر کش

In [0]:
main_gru_model.save("char_gru.h5")

# Model (LSTM)

In [0]:
def create_model_lstm(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE):
  model = keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
  model.add(tf.keras.layers.Dense(vocab_size))
  
  return model

In [25]:
lstm_model = create_model_lstm()
lstm_model.summary()
lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
lstm_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense_4 (Dense)              (64, None, 48)            49200     
Total params: 13,701,168
Trainable params: 13,701,168
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           12288     
____________________

In [0]:
history = lstm_model.fit(dataset, epochs=50)

Epoch 1/50
42/42 [==============================] - 141s 3s/step - loss: 12.8320 - accuracy: 0.2125
Epoch 2/50
42/42 [==============================] - 142s 3s/step - loss: 3.4960 - accuracy: 0.2607
Epoch 3/50
42/42 [==============================] - 142s 3s/step - loss: 2.6518 - accuracy: 0.2998
Epoch 4/50
42/42 [==============================] - 142s 3s/step - loss: 2.4473 - accuracy: 0.3255
Epoch 5/50
42/42 [==============================] - 142s 3s/step - loss: 2.3315 - accuracy: 0.3431
Epoch 6/50
42/42 [==============================] - 143s 3s/step - loss: 2.2422 - accuracy: 0.3604
Epoch 7/50
42/42 [==============================] - 144s 3s/step - loss: 2.2031 - accuracy: 0.3704
Epoch 8/50
42/42 [==============================] - 144s 3s/step - loss: 2.1097 - accuracy: 0.3900
Epoch 9/50
42/42 [==============================] - 144s 3s/step - loss: 2.0423 - accuracy: 0.4036
Epoch 10/50
42/42 [==============================] - 144s 3s/step - loss: 2.0018 - accuracy: 0.4141
Epoch 11

In [0]:
main_lstm_model = create_model_lstm(batch_size=1)
main_lstm_model.set_weights(lstm_model.get_weights())
main_lstm_model.summary()

In [0]:
# main_lstm_model = tf.keras.models.load_model('char_lstm.h5')
# main_lstm_model.summary()

In [0]:
print(generate_text(main_lstm_model, start_string=u"که ایران چوباغیست خرم بهار", temperature=1))

In [0]:
main_lstm_model.save("char_lstm.h5")